In [ ]:
import pyterrier as pt
import pandas as pd
from tira.third_party_integrations import ensure_pyterrier_is_loaded

ensure_pyterrier_is_loaded()

ds_id = "spot-check-20251122-training"
dataset = pt.datasets.get_dataset(f"irds:ir-lab-wise-2025/{ds_id}")
topics = dataset.get_topics("title")

Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [4]:
import os
print(os.getcwd())

/workspaces/ks-golden-retrievals


In [19]:
INDEX_BASE = "/workspaces/ks-golden-retrievals/output/indexes/spot-check-20251122-training-on-default_text-with-bo1"
INDEX_STEM = "/workspaces/ks-golden-retrievals/output/indexes/spot-check-20251122-training-on-default_text-porter-terrier-stop"

index_baseline = pt.IndexFactory.of(INDEX_BASE)
index_stem = pt.IndexFactory.of(INDEX_STEM)


In [20]:
retr_baseline = pt.terrier.Retriever(index_baseline, wmodel="BM25")
retr_stem = pt.terrier.Retriever(index_stem, wmodel="BM25")

query = topics.iloc[0:1]  # exakt eine Query

In [21]:
res_base = retr_baseline(query)
res_stem = retr_stem(query)

In [23]:
print("Baseline terms:",
      index_baseline.getLexicon().numberOfEntries())
print("Stop terms:",
      index_stem.getLexicon().numberOfEntries())

Baseline terms: 8183
Stop terms: 8163


In [24]:
df_scores = pd.DataFrame({
    "docno": res_base["docno"].head(10).values,
    "score_baseline": res_base["score"].head(10).values,
    "stop": res_stem.set_index("docno")
                                   .loc[res_base["docno"].head(10)]
                                   ["score"]
                                   .values
})

df_scores

,docno,score_baseline,stop
0,e2e37f514110c0b3b7548903495348fac3db79d29df27a...,11.834171,11.838878
1,5bab34028f894e2aa2f94a29e331c05f215a55e8c208f0...,11.148937,11.130674
2,ef182a141e53e8b9347cceac30d41c1fd9cfaa4aca170e...,10.279988,10.259520
3,a18557d59ff2214a603ca6c27f59d08b176e6a17c4171d...,10.180116,10.146613
4,e1845ad7ca451126a8ddaf0f5e9877239222da3e3ab0d3...,10.111481,10.081335
5,e00a412c07e34bca044b729bc218e92edca60336f7ecc1...,10.073976,10.035708
6,38f4a96022cda7b2e85db2015f2949b340bf4116d76ea7...,9.845102,9.857148
7,7c8c5551ad96a59f8074e3e7a076377ea9a8fd43f3ab15...,9.775547,9.775408
8,8a9be855b709647ee83459e3fad1381a7f23443f2cc46a...,9.705914,9.726996
9,1ddb8e93d2259c719bd72ea589132b4b3368e72cb2dac6...,9.555875,9.555393
